# <font color='CC3D3D'> Log Feature Data Clansing
    
### 1. Data Merge
- <span style="color:green">**Master_df.csv** </span>와 <span style="color:green">**loan_with_new_feats.csv, loan_path_segment.csv** </span>를 Merge

### 2. MissingValue Preprocessing
- ['application_id','loanapply_insert_time','loan_rate','credit_score','loan_limit', 'existing_loan_amt','desired_amount','yearly_income','existing_loan_cnt']를 가지고 Knn Imputer를 가지고 결측치 대체
- Segment 결측치에 대해서만 새로운 Segment 부여

### 3. Deployment CSV
- Clansing한 <span style="color:blue">**log_feature_clansing.csv** </span>를 생성

# Import

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from pycaret.classification import *

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
import klib

plt.rc("font", family = 'Malgun Gothic')
plt.rcParams["figure.figsize"] = (12,8)

# Data Load

In [2]:
log_feature = pd.read_csv('../Data/loan_with_new_feats.csv')
log_segment = pd.read_csv('../Data/loan_path_segment.csv')
master_df = pd.read_csv('../Data/Master_df.csv')

In [3]:
display(log_feature)

,application_id,loanapply_insert_time,is_applied,user_id,date_cd,event,loan_cnt,duration,visit
0,248576,2022-05-21 23:41:09,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0
1,248576,2022-05-21 23:41:09,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0
2,248576,2022-05-21 23:41:09,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0
3,248576,2022-05-21 23:41:09,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0
4,248576,2022-05-21 23:41:10,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0
...,...,...,...,...,...,...,...,...,...
13527245,1543548,2022-05-10 20:40:55,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN
13527246,1543548,2022-05-10 20:40:55,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN
13527247,1543548,2022-05-10 20:41:07,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN
13527248,1543548,2022-05-10 20:41:10,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN


In [4]:
display(log_segment)

,application_id,loanapply_insert_time,is_applied,user_id,labels
0,1748340,2022-06-07 13:05:41,0.0,430982,0.0
1,1748340,2022-06-07 13:05:41,0.0,430982,0.0
2,1748340,2022-06-07 13:05:41,0.0,430982,0.0
3,1748340,2022-06-07 13:05:41,0.0,430982,0.0
4,1748340,2022-06-07 13:05:41,0.0,430982,0.0
...,...,...,...,...,...
13545166,1513515,2022-06-03 12:07:49,1.0,31658,1.0
13545167,1513515,2022-06-03 12:07:50,1.0,31658,1.0
13545168,1513515,2022-06-03 12:07:51,1.0,31658,1.0
13545169,1513515,2022-06-03 12:07:49,1.0,31658,1.0


In [5]:
display(master_df)

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,user_id,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_year,enter_month,enter_birth,month,birth_year,gender,month_day,k_lending_rate,us_lending_rate,btc_price,btc_variance,eth_price,eth_variance,lunc_price,lunc_variance,Close,Open,High,Low,Volume,Change,실업자,실업률,연령대,소득수준,현재생활형편CSI_성별,현재생활형편CSI_나이,현재생활형편CSI_소득,현재가계부채CSI,가계부채전망CSI,소비심리지수,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527245,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN
13527246,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN
13527247,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN
13527248,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,

In [6]:
log_feature.isna().sum()

application_id                 0
loanapply_insert_time          0
is_applied               3257239
user_id                        0
date_cd                        0
event                     703483
loan_cnt                  703483
duration                  703483
visit                     703483
dtype: int64

In [7]:
log_segment.isna().sum()

application_id                0
loanapply_insert_time         0
is_applied               111972
user_id                       0
labels                   596630
dtype: int64

In [8]:
master_df.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
user_id                                      0
insert_time                                  0
credit_score                                 0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
enter_year                                   0
enter_month                                  0
enter_birth  

In [9]:
log_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527250 entries, 0 to 13527249
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   loanapply_insert_time  object 
 2   is_applied             float64
 3   user_id                int64  
 4   date_cd                object 
 5   event                  float64
 6   loan_cnt               float64
 7   duration               float64
 8   visit                  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 928.8+ MB


In [10]:
log_segment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13545171 entries, 0 to 13545170
Data columns (total 5 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   loanapply_insert_time  object 
 2   is_applied             float64
 3   user_id                int64  
 4   labels                 float64
dtypes: float64(2), int64(2), object(1)
memory usage: 516.7+ MB


In [11]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527250 entries, 0 to 13527249
Data columns (total 51 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   application_id                       int64  
 1   loanapply_insert_time                object 
 2   bank_id                              int64  
 3   product_id                           int64  
 4   loan_limit                           float64
 5   loan_rate                            float64
 6   user_id                              float64
 7   insert_time                          object 
 8   credit_score                         float64
 9   yearly_income                        float64
 10  income_type                          object 
 11  employment_type                      object 
 12  houseown_type                        object 
 13  desired_amount                       float64
 14  purpose                              object 
 15  personal_rehabilitation_yn    

# Data Merge

In [12]:
log_df = pd.merge(log_feature,log_segment[['application_id','loanapply_insert_time','labels']],
         on=['application_id','loanapply_insert_time'],how='left').drop_duplicates().reset_index(drop=True);log_df

,application_id,loanapply_insert_time,is_applied,user_id,date_cd,event,loan_cnt,duration,visit,labels
0,248576,2022-05-21 23:41:09,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0,3.0
1,248576,2022-05-21 23:41:10,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0,3.0
2,248576,2022-05-21 23:41:11,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0,3.0
3,248576,2022-05-21 23:41:14,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0,3.0
4,248576,2022-05-21 23:41:15,0.0,9,2022-05-21 00:00:00,3.0,0.0,354.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
4565098,1543548,2022-05-10 20:40:54,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN
4565099,1543548,2022-05-10 20:40:55,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN
4565100,1543548,2022-05-10 20:41:07,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN
4565101,1543548,2022-05-10 20:41:10,0.0,879697,2022-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN


In [13]:
log_df.drop(['is_applied','user_id','date_cd'],axis=1,inplace=True)

In [14]:
log_df.columns = ['application_id','loanapply_insert_time','event','loan_cnt','duration','visit','segment']

In [15]:
log_df

,application_id,loanapply_insert_time,event,loan_cnt,duration,visit,segment
0,248576,2022-05-21 23:41:09,3.0,0.0,354.0,1.0,3.0
1,248576,2022-05-21 23:41:10,3.0,0.0,354.0,1.0,3.0
2,248576,2022-05-21 23:41:11,3.0,0.0,354.0,1.0,3.0
3,248576,2022-05-21 23:41:14,3.0,0.0,354.0,1.0,3.0
4,248576,2022-05-21 23:41:15,3.0,0.0,354.0,1.0,3.0
...,...,...,...,...,...,...,...
4565098,1543548,2022-05-10 20:40:54,NaN,NaN,NaN,NaN,NaN
4565099,1543548,2022-05-10 20:40:55,NaN,NaN,NaN,NaN,NaN
4565100,1543548,2022-05-10 20:41:07,NaN,NaN,NaN,NaN,NaN
4565101,1543548,2022-05-10 20:41:10,NaN,NaN,NaN,NaN,NaN


# Missing Value

In [16]:
log_df.isna().sum()

application_id                0
loanapply_insert_time         0
event                    216644
loan_cnt                 216644
duration                 216644
visit                    216644
segment                  179732
dtype: int64

## Knn imputer

### Data Generation

In [17]:
md = master_df[['application_id','loanapply_insert_time','loan_rate','credit_score','loan_limit',
               'existing_loan_amt','desired_amount','yearly_income','existing_loan_cnt']].drop_duplicates()

In [18]:
md_df = md.groupby(['application_id','loanapply_insert_time']).mean().reset_index()
md_df

,application_id,loanapply_insert_time,loan_rate,credit_score,loan_limit,existing_loan_amt,desired_amount,yearly_income,existing_loan_cnt
0,1,2022-05-06 11:05:42,17.7,580.0,1.900000e+07,32800000.0,30000000.0,40000000.0,4.2
1,1,2022-05-06 11:05:43,18.5,580.0,1.850000e+07,32800000.0,30000000.0,40000000.0,4.2
2,1,2022-05-06 11:05:44,18.9,580.0,3.200000e+07,32800000.0,30000000.0,40000000.0,4.2
3,1,2022-05-06 11:05:56,17.1,580.0,1.900000e+07,32800000.0,30000000.0,40000000.0,4.2
4,1,2022-05-06 11:05:57,14.8,580.0,8.000000e+06,32800000.0,30000000.0,40000000.0,4.2
...,...,...,...,...,...,...,...,...,...
4293458,2167822,2022-06-29 14:08:35,15.6,570.0,1.050000e+07,6000000.0,7000000.0,13000000.0,2.0
4293459,2167822,2022-06-29 14:08:36,18.3,570.0,1.100000e+07,6000000.0,7000000.0,13000000.0,2.0
4293460,2167826,2022-03-25 11:57:49,18.5,560.0,1.566667e+07,79000000.0,5000000.0,32000000.0,10.0
4293461,2167826,2022-03-25 11:57:51,17.4,560.0,1.300000e+07,79000000.0,5000000.0,32000000.0,10.0


In [19]:
knn_log = log_df.iloc[:,:-1]

In [20]:
knn_df = pd.merge(knn_log,md_df,on=['application_id','loanapply_insert_time'],how='left')
knn_df

,application_id,loanapply_insert_time,event,loan_cnt,duration,visit,loan_rate,credit_score,loan_limit,existing_loan_amt,desired_amount,yearly_income,existing_loan_cnt
0,248576,2022-05-21 23:41:09,3.0,0.0,354.0,1.0,9.000000,810.0,1.700000e+07,17800000.0,15000000.0,29000000.0,1.0
1,248576,2022-05-21 23:41:10,3.0,0.0,354.0,1.0,12.927273,810.0,2.118182e+07,17800000.0,15000000.0,29000000.0,1.0
2,248576,2022-05-21 23:41:11,3.0,0.0,354.0,1.0,13.625000,810.0,2.200000e+07,17800000.0,15000000.0,29000000.0,1.0
3,248576,2022-05-21 23:41:14,3.0,0.0,354.0,1.0,7.800000,810.0,5.000000e+06,17800000.0,15000000.0,29000000.0,1.0
4,248576,2022-05-21 23:41:15,3.0,0.0,354.0,1.0,13.100000,810.0,3.700000e+07,17800000.0,15000000.0,29000000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4565098,1543548,2022-05-10 20:40:54,NaN,NaN,NaN,NaN,11.177778,894.0,3.211111e+07,171000000.0,200000000.0,70000000.0,1.8
4565099,1543548,2022-05-10 20:40:55,NaN,NaN,NaN,NaN,9.285714,894.0,4.442857e+07,171000000.0,200000000.0,70000000.0,1.8
4565100,1543548,2022-05-10 20:41:07,NaN,NaN,NaN,NaN,5.400000,894.0,3.000000e+06,171000000.0,200000000.0,70000000.0,1.8
4565101,1543548,2022-05-10 20:41:10,NaN,NaN,NaN,NaN,10.600000,894.0,5.000000e+07,171000000.0,200000000.0,70000000.0,1.8


In [21]:
knn_df.isna().sum()

application_id                0
loanapply_insert_time         0
event                    216644
loan_cnt                 216644
duration                 216644
visit                    216644
loan_rate                     0
credit_score                  0
loan_limit                    0
existing_loan_amt             0
desired_amount                0
yearly_income                 0
existing_loan_cnt             0
dtype: int64

In [22]:
knn_df.drop(['application_id','loanapply_insert_time'],axis=1,inplace=True)

In [23]:
knn_df_train = knn_df[knn_df.event.notna()].reset_index(drop=True);knn_df_train

,event,loan_cnt,duration,visit,loan_rate,credit_score,loan_limit,existing_loan_amt,desired_amount,yearly_income,existing_loan_cnt
0,3.0,0.0,354.0,1.0,9.000000,810.0,1.700000e+07,17800000.0,15000000.0,29000000.0,1.0
1,3.0,0.0,354.0,1.0,12.927273,810.0,2.118182e+07,17800000.0,15000000.0,29000000.0,1.0
2,3.0,0.0,354.0,1.0,13.625000,810.0,2.200000e+07,17800000.0,15000000.0,29000000.0,1.0
3,3.0,0.0,354.0,1.0,7.800000,810.0,5.000000e+06,17800000.0,15000000.0,29000000.0,1.0
4,3.0,0.0,354.0,1.0,13.100000,810.0,3.700000e+07,17800000.0,15000000.0,29000000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4348454,11.0,2.0,400.0,1.0,9.400000,610.0,9.000000e+06,20000000.0,10000000.0,37000000.0,3.0
4348455,11.0,2.0,400.0,1.0,16.500000,610.0,1.000000e+07,20000000.0,10000000.0,37000000.0,3.0
4348456,11.0,2.0,400.0,1.0,8.400000,610.0,9.000000e+06,20000000.0,10000000.0,37000000.0,3.0
4348457,11.0,2.0,400.0,1.0,11.300000,610.0,1.000000e+07,20000000.0,10000000.0,37000000.0,3.0


In [24]:
scaler = MinMaxScaler().fit(knn_df_train)

In [25]:
train_knn = scaler.transform(knn_df_train)

In [26]:
test_knn = scaler.transform(knn_df)

In [27]:
imputer = KNNImputer(n_neighbors = 5).fit(train_knn)

In [28]:
knn_test = imputer.transform(test_knn)

In [29]:
knn_test_scaler = scaler.inverse_transform(knn_test)

In [30]:
knn_df = pd.DataFrame(knn_test_scaler,columns = knn_df.columns);knn_df

,event,loan_cnt,duration,visit,loan_rate,credit_score,loan_limit,existing_loan_amt,desired_amount,yearly_income,existing_loan_cnt
0,3.0,0.0,354.0,1.0,9.000000,810.0,1.700000e+07,17800000.0,15000000.0,29000000.0,1.0
1,3.0,0.0,354.0,1.0,12.927273,810.0,2.118182e+07,17800000.0,15000000.0,29000000.0,1.0
2,3.0,0.0,354.0,1.0,13.625000,810.0,2.200000e+07,17800000.0,15000000.0,29000000.0,1.0
3,3.0,0.0,354.0,1.0,7.800000,810.0,5.000000e+06,17800000.0,15000000.0,29000000.0,1.0
4,3.0,0.0,354.0,1.0,13.100000,810.0,3.700000e+07,17800000.0,15000000.0,29000000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4565098,5.0,2.4,181.8,1.0,11.177778,894.0,3.211111e+07,171000000.0,200000000.0,70000000.0,1.8
4565099,6.0,4.0,167.6,1.2,9.285714,894.0,4.442857e+07,171000000.0,200000000.0,70000000.0,1.8
4565100,6.0,4.6,243.6,1.0,5.400000,894.0,3.000000e+06,171000000.0,200000000.0,70000000.0,1.8
4565101,5.0,3.0,115.4,1.2,10.600000,894.0,5.000000e+07,171000000.0,200000000.0,70000000.0,1.8


In [31]:
log_df

,application_id,loanapply_insert_time,event,loan_cnt,duration,visit,segment
0,248576,2022-05-21 23:41:09,3.0,0.0,354.0,1.0,3.0
1,248576,2022-05-21 23:41:10,3.0,0.0,354.0,1.0,3.0
2,248576,2022-05-21 23:41:11,3.0,0.0,354.0,1.0,3.0
3,248576,2022-05-21 23:41:14,3.0,0.0,354.0,1.0,3.0
4,248576,2022-05-21 23:41:15,3.0,0.0,354.0,1.0,3.0
...,...,...,...,...,...,...,...
4565098,1543548,2022-05-10 20:40:54,NaN,NaN,NaN,NaN,NaN
4565099,1543548,2022-05-10 20:40:55,NaN,NaN,NaN,NaN,NaN
4565100,1543548,2022-05-10 20:41:07,NaN,NaN,NaN,NaN,NaN
4565101,1543548,2022-05-10 20:41:10,NaN,NaN,NaN,NaN,NaN


In [32]:
log_df.event = knn_df.event
log_df.loan_cnt = knn_df.loan_cnt
log_df.duration = knn_df.duration
log_df.visit = knn_df.visit

In [33]:
log_df.isna().sum()

application_id                0
loanapply_insert_time         0
event                         0
loan_cnt                      0
duration                      0
visit                         0
segment                  179732
dtype: int64

## segment

In [34]:
log_df.segment.unique()

array([ 3.,  4.,  0., nan,  1.,  5.,  2.])

In [35]:
log_df.segment.fillna(6,inplace=True)

In [36]:
log_df.isna().sum()

application_id           0
loanapply_insert_time    0
event                    0
loan_cnt                 0
duration                 0
visit                    0
segment                  0
dtype: int64

# Data Generation

In [37]:
log = pd.merge(master_df,log_df,on=['application_id','loanapply_insert_time'],how='left').drop_duplicates(['application_id','loanapply_insert_time','bank_id','product_id'])
log

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,user_id,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,enter_year,enter_month,enter_birth,month,birth_year,gender,month_day,k_lending_rate,us_lending_rate,btc_price,btc_variance,eth_price,eth_variance,lunc_price,lunc_variance,Close,Open,High,Low,Volume,Change,실업자,실업률,연령대,소득수준,현재생활형편CSI_성별,현재생활형편CSI_나이,현재생활형편CSI_소득,현재가계부채CSI,가계부채전망CSI,소비심리지수,is_applied,event,loan_cnt,duration,visit,segment
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN,18.0,12.0,861.0,3.0,0.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN,18.0,12.0,861.0,3.0,0.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN,18.0,12.0,861.0,3.0,0.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN,18.0,12.0,861.0,3.0,0.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,430982.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,2022.0,2.0,26.0,2022-06,1996.0,1.0,2022-06-07,1.75,1.0,39544504.0,0.0408,2347000.0,0.0224,0.000081,0.0174,2670.65,2679.57,2681.51,2663.00,562470000,0.0044,156,7.8,40세미만,200-300만원,89,94,85,103.0,100.0,92.17,NaN,18.0,12.0,861.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15090062,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN,6.0,3.0,166.0,1.0,1.0
15090063,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN,6.0,3.0,166.0,1.0,1.0
15090064,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,478802.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,정규직,기타가족소유,50000000.0,대환대출,0.0,0.0,2.0,56000000.0,2019.0,3.0,54.0,2022-06,1965.0,1.0,2022-06-03,1.75,1.0,38376912.0,0.0134,2310990.0,0.0005,0.000101,0.0202,2658.99,2670.74,2674.00,2653.59,525260000,-0.0100,68,1.8,50-60세,300-400만원,89,89,87,103.0,100.0,91.50,NaN,6.0,3.0,166.0

In [38]:
log.isna().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
user_id                                      0
insert_time                                  0
credit_score                                 0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
enter_year                                   0
enter_month                                  0
enter_birth  

# Data Split

In [39]:
log_feature = log[['event','loan_cnt','duration','visit','segment']]

In [40]:
log_feature

,event,loan_cnt,duration,visit,segment
0,18.0,12.0,861.0,3.0,0.0
1,18.0,12.0,861.0,3.0,0.0
2,18.0,12.0,861.0,3.0,0.0
3,18.0,12.0,861.0,3.0,0.0
4,18.0,12.0,861.0,3.0,0.0
...,...,...,...,...,...
15090062,6.0,3.0,166.0,1.0,1.0
15090063,6.0,3.0,166.0,1.0,1.0
15090064,6.0,3.0,166.0,1.0,1.0
15090065,6.0,3.0,166.0,1.0,1.0


# Deployment CSV

In [41]:
log_feature.to_csv('../Data/log_feature_clansing.csv',index=False)